In [3]:
from tqdm.notebook import tqdm

import json
import pickle
import string

from langchain import PromptTemplate

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import transformers
import torch

from pdb import set_trace

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

In [4]:
tasks = [
    
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-free',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-complex-free',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-complex-force',
    'wdcproducts-80cc-seen-sampled-250-gs-2_general-simple-free',
    'wdcproducts-80cc-seen-sampled-250-gs-2_general-complex-free',
    'wdcproducts-80cc-seen-sampled-250-gs-2_general-simple-force',
    'wdcproducts-80cc-seen-sampled-250-gs-2_general-complex-force',

    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-rules-heterogeneity',
    'wdcproducts-80cc-seen-sampled-250-gs-2_domain-simple-force-rules-gen-handpicked',
    
    'wdcproducts-80cc-seen-sampled-250-gs-2_relatedwork-simple',
    'wdcproducts-80cc-seen-sampled-250-gs-2_relatedwork-complex',
    
    'abt-buy-sampled-gs_domain-simple-free',
    'abt-buy-sampled-gs_domain-complex-free',
    'abt-buy-sampled-gs_domain-simple-force',
    'abt-buy-sampled-gs_domain-complex-force',
    'abt-buy-sampled-gs_general-simple-free',
    'abt-buy-sampled-gs_general-complex-free',
    'abt-buy-sampled-gs_general-simple-force',
    'abt-buy-sampled-gs_general-complex-force',
    
    'abt-buy-sampled-gs_relatedwork-simple',
    'abt-buy-sampled-gs_relatedwork-complex',
    
    'abt-buy-sampled-gs_domain-simple-force-rules-heterogeneity',
    'abt-buy-sampled-gs_domain-simple-force-rules-gen-handpicked',
    
    'amazon-google-sampled-gs_domain-simple-free',
    'amazon-google-sampled-gs_domain-complex-free',
    'amazon-google-sampled-gs_domain-simple-force',
    'amazon-google-sampled-gs_domain-complex-force',
    'amazon-google-sampled-gs_general-simple-free',
    'amazon-google-sampled-gs_general-complex-free',
    'amazon-google-sampled-gs_general-simple-force',
    'amazon-google-sampled-gs_general-complex-force',
    
    'amazon-google-sampled-gs_relatedwork-simple',
    'amazon-google-sampled-gs_relatedwork-complex',
    
    'amazon-google-sampled-gs_domain-simple-force-rules-heterogeneity',
    'amazon-google-sampled-gs_domain-simple-force-rules-gen-handpicked',
    
    'walmart-amazon-sampled-gs_domain-simple-free',
    'walmart-amazon-sampled-gs_domain-complex-free',
    'walmart-amazon-sampled-gs_domain-simple-force',
    'walmart-amazon-sampled-gs_domain-complex-force',
    'walmart-amazon-sampled-gs_general-simple-free',
    'walmart-amazon-sampled-gs_general-complex-free',
    'walmart-amazon-sampled-gs_general-simple-force',
    'walmart-amazon-sampled-gs_general-complex-force',
    
    'walmart-amazon-sampled-gs_relatedwork-simple',
    'walmart-amazon-sampled-gs_relatedwork-complex',
    
    'walmart-amazon-sampled-gs_domain-simple-force-rules-heterogeneity',
    'walmart-amazon-sampled-gs_domain-simple-force-rules-gen-handpicked',
    
    'dblp-scholar-sampled-gs_domain-simple-free',
    'dblp-scholar-sampled-gs_domain-complex-free',
    'dblp-scholar-sampled-gs_domain-simple-force',
    'dblp-scholar-sampled-gs_domain-complex-force',
    'dblp-scholar-sampled-gs_general-simple-free',
    'dblp-scholar-sampled-gs_general-complex-free',
    'dblp-scholar-sampled-gs_general-simple-force',
    'dblp-scholar-sampled-gs_general-complex-force',
    
    'dblp-scholar-sampled-gs_relatedwork-simple',
    'dblp-scholar-sampled-gs_relatedwork-complex',
    
    'dblp-scholar-sampled-gs_domain-simple-force-rules-heterogeneity',
    'dblp-scholar-sampled-gs_domain-simple-force-rules-gen-handpicked',
    
    'dblp-acm-sampled-gs_domain-simple-free',
    'dblp-acm-sampled-gs_domain-complex-free',
    'dblp-acm-sampled-gs_domain-simple-force',
    'dblp-acm-sampled-gs_domain-complex-force',
    'dblp-acm-sampled-gs_general-simple-free',
    'dblp-acm-sampled-gs_general-complex-free',
    'dblp-acm-sampled-gs_general-simple-force',
    'dblp-acm-sampled-gs_general-complex-force',
    
    'dblp-acm-sampled-gs_relatedwork-simple',
    'dblp-acm-sampled-gs_relatedwork-complex',
    
    'dblp-acm-sampled-gs_domain-simple-force-rules-heterogeneity',
    'dblp-acm-sampled-gs_domain-simple-force-rules-gen-handpicked',
    ]

In [5]:
model_id = "upstage/Llama-2-70b-instruct-v2"
model_name = "Llama-2-70b-instruct-v2"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_8bit=True,
    rope_scaling={"type": "dynamic", "factor": 2} # allows handling of longer inputs
)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

ImportError: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
attributes = ['default']

for task in tasks:
    for attribute in attributes:
        
        
        # open the JSON file in read mode
        with open(f'../tasks/{task}.json', 'r') as f:
            # load the JSON data from the file and convert it into a dictionary
            task_dict = json.load(f)
        
        # Create LangChain PromptTemplate

        template = """{task_prefix}{input_string}"""
        prompt = PromptTemplate(
                template=template,
                input_variables=['task_prefix', 'input_string']
        )
        
        prompts = []
        
        
        if attribute == 'default':
            for example in task_dict['examples']:
                text_prompt = prompt.format(task_prefix=task_dict['task_prefix'], input_string=example['input'])
                text_prompt = f"### User: {text_prompt}\n\n### Assistant:\n"
                prompts.append(text_prompt)
                
        targets = [example['target_scores'] for example in task_dict['examples']]
        
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
        sequences = pipeline(
            prompts,
            #streamer=streamer,
            use_cache=True,
            max_new_tokens=100
        )               

        # Do some data wrangling to format target and preds to match squad V2
        answers = []
        predictions = []
        truth = []
        unclear_answers = 0
        num_long_answers = 0
        for i in range(len(targets)):
            if targets[i]['Yes'] == 1:
                truth.append(1)
            else:
                truth.append(0)

            processed_pred = sequences[i][0]['generated_text'].replace(f'{prompts[i]}', '')
            answers.append(processed_pred)            
                    
            # handle yes/no answers
            
            processed_pred = processed_pred.strip().translate(str.maketrans('', '', string.punctuation)).lower()

            if processed_pred != 'yes' and processed_pred != 'no':
                print(f'Overlong Answer: {processed_pred}')
                num_long_answers += 1
            if 'yes' in processed_pred:
                processed_pred = 'yes'
            elif 'no' in processed_pred:
                processed_pred = 'no'
            else:
                processed_pred = 'no'
                unclear_answers += 1

            if processed_pred == 'yes':
                predictions.append(1)
            elif processed_pred == 'no':
                predictions.append(0)
        
        # save the prompts
        with open(f'../prompts/{task}_{attribute}_{model_name}_run-1.pickle', 'wb') as handle:
            pickle.dump(prompts, handle, protocol=pickle.HIGHEST_PROTOCOL)
        # save the answers
        with open(f'../answers/{task}_{attribute}_{model_name}_run-1.pickle', 'wb') as handle:
            pickle.dump(answers, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
        precision = precision_score(truth, predictions)
        recall = recall_score(truth, predictions)
        f1 = f1_score(truth, predictions)
        accuracy = accuracy_score(truth, predictions)
        

        results = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1': f1
        }

        with open(f"../results/result_{task}_{attribute}_{model_name}_run-1.json", "w") as outfile:
            json.dump(results, outfile, indent=2)